# Debugger and Profiler

In [1]:
# install dependencies
!pip install smdebug

    100% |████████████████████████████████| 276kB 3.4MB/s ta 0:00:01
    100% |████████████████████████████████| 163kB 17.3MB/s ta 0:00:01
    100% |████████████████████████████████| 13.4MB 2.5MB/s eta 0:00:01 0% |▏                               | 51kB 17.0MB/s eta 0:00:01    8% |██▋                             | 1.1MB 21.1MB/s eta 0:00:01    16% |█████▎                          | 2.2MB 23.7MB/s eta 0:00:01    32% |██████████▌                     | 4.4MB 23.1MB/s eta 0:00:01    41% |█████████████▏                  | 5.5MB 23.3MB/s eta 0:00:01    49% |███████████████▊                | 6.6MB 24.1MB/s eta 0:00:01    57% |██████████████████▎             | 7.6MB 23.0MB/s eta 0:00:01    65% |████████████████████▉           | 8.7MB 22.9MB/s eta 0:00:01    89% |████████████████████████████▋   | 12.0MB 23.5MB/s eta 0:00:01
    100% |████████████████████████████████| 133kB 17.6MB/s ta 0:00:01
    100% |████████████████████████████████| 92kB 15.4MB/s ta 0:00:01
    100% |█████████████████████████

In [2]:
hyperparameters = {
    "batch_size": 2048,
    "gpu": True,
    "epoch": 2,
    "model": "resnet50",
}

In [6]:
!pip install sagemaker

    100% |████████████████████████████████| 491kB 15.8MB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 13.6MB/s ta 0:00:01
    100% |████████████████████████████████| 40kB 10.8MB/s ta 0:00:01
    100% |████████████████████████████████| 81kB 14.9MB/s ta 0:00:01
    100% |████████████████████████████████| 92kB 16.0MB/s ta 0:00:01
    100% |████████████████████████████████| 112kB 9.9MB/s eta 0:00:01   67% |█████████████████████▍          | 71kB 20.5MB/s eta 0:00:01
    100% |████████████████████████████████| 71kB 15.3MB/s ta 0:00:01
  Running setup.py bdist_wheel for sagemaker ... done
  Stored in directory: /root/.cache/pip/wheels/3d/73/a8/77fe69fb57f426d18fb8a1d27721e0711e18392c18b3dc0107
  Running setup.py bdist_wheel for protobuf3-to-dict ... done
  Stored in directory: /root/.cache/pip/wheels/37/42/d8/1609d310cabebc2cf60eca038fa2b0c8503412963734a6fc31
Successfully built sagemaker protobuf3-to-dict
  Found existing installation: packaging 16.8
    Uninstalling packagin

In [7]:
from sagemaker.debugger import Rule, ProfilerRule, rule_configs

rules = [
    Rule.sagemaker(rule_configs.vanishing_gradient()),
    Rule.sagemaker(rule_configs.overfit()),
    Rule.sagemaker(rule_configs.overtraining()),
    Rule.sagemaker(rule_configs.poor_weight_initialization()),
    Rule.sagemaker(rule_configs.loss_not_decreasing()),
    ProfilerRule.sagemaker(rule_configs.LowGPUUtilization()),
    ProfilerRule.sagemaker(rule_configs.ProfilerReport())
]

AttributeError: module 'google.protobuf.internal.containers' has no attribute 'MutableMapping'

In [ ]:
from sagemaker.debugger import DebuggerHookConfig, ProfilerConfig, FrameworkProfile

#TODO: Can you create the profiler and debugger configs
profiler_config = ProfilerConfig(system_monitor_interval_millis=500, framework_profile_params=FrameworkProfile(num_steps=10))
debugger_config = DebuggerHookConfig(hook_parameters={'train.save_interval':'100', 'eval.save_interval':'10'})

In [ ]:
import sagemaker
from sagemaker.pytorch import PyTorch
from sagemaker import get_execution_role

#TODO: Create the estimator to train your model
estimator = PyTorch(
    role=get_execution_role(),
    instance_count=1,
    instance_type='ml.m5.xlarge',
    entry_point='',
    framework_version=,
    py_version=,
    hyperparameters=,
    # debugger & profiler Parameters
    rules=rules,
    debugger_hook_config=debugger_config,
    profiler_config=profiler_config
    )

In [ ]:
estimator.fit(wait=True)

In [ ]:
import boto3

session = boto3.session.Session()
region = session.region_name

training_job_name = estimator.latest_training_job.name
print(f"Training jobname: {training_job_name}")
print(f"Region: {region}")

In [ ]:
from smdebug.trials import create_trial
from smdebug.core.modes import ModeKeys

trial = create_trial(estimator.latest_job_debugger_artifacts_path())

In [ ]:
# TODO: Can you print the names of all the tensors that were tracked
# TODO: Can you print the number of datapoints for one of those tensors
# for both train and eval mode

In [ ]:
from smdebug.profiler.analysis.notebook_utils.training_job import TrainingJob

tj = TrainingJob(training_job_name, region)
tj.wait_for_sys_profiling_data_to_be_available()

In [ ]:
from smdebug.profiler.analysis.notebook_utils.timeline_charts import TimelineCharts

system_metrics_reader = tj.get_systems_metrics_reader()
system_metrics_reader.refresh_event_file_list()

view_timeline_charts = TimelineCharts(
    system_metrics_reader,
    framework_metrics_reader=None,
    select_dimensions=["CPU", "GPU"],
    select_events=["total"],
)

In [ ]:
rule_output_path = estimator.output_path + estimator.latest_training_job.job_name + "/rule-output"
print(f"You will find the profiler report in {rule_output_path}")

In [ ]:
! aws s3 ls {rule_output_path} --recursive

In [ ]:
! aws s3 cp {rule_output_path} ./ --recursive

In [ ]:
import os

# get the autogenerated folder name of profiler report
profiler_report_name = [
    rule["RuleConfigurationName"]
    for rule in estimator.latest_training_job.rule_job_summary()
    if "Profiler" in rule["RuleConfigurationName"]
][0]

In [ ]:
import IPython

IPython.display.HTML(filename=profiler_report_name + "/profiler-output/profiler-report.html")